In [2]:
import json
import pandas as pd

In [5]:
with open("../data/clinical.cases_selection.2019-10-30.json") as f:
    j = json.load(f)

In [22]:
dd = []
for jj in j:
    ddx = {}
    try:
        ddx.update(jj["diagnoses"][0])
    except KeyError:
        continue
    ddx.update(jj["demographic"])
    dd.append(ddx)

In [23]:
df = pd.DataFrame.from_records(dd)

In [25]:
df.head()

,year_of_diagnosis,classification_of_tumor,last_known_disease_status,updated_datetime,primary_diagnosis,submitter_id,tumor_stage,age_at_diagnosis,morphology,days_to_last_known_disease_status,...,vascular_invasion_present,micropapillary_features,anaplasia_present_type,medulloblastoma_molecular_classification,cog_renal_stage,metastasis_at_diagnosis_site,gleason_grade_group,cause_of_death,weeks_gestation_at_birth,premature_at_birth
0,2007.0,not reported,not reported,2019-07-31T23:10:30.924622-05:00,"Clear cell adenocarcinoma, NOS",TCGA-A3-3336_demographic,stage i,27538.0,8310/3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2005.0,not reported,not reported,2019-07-31T23:07:23.969026-05:00,"Clear cell adenocarcinoma, NOS",TCGA-CZ-4854_demographic,stage i,24876.0,8310/3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005.0,not reported,not reported,2019-07-31T23:24:17.681086-05:00,"Clear cell adenocarcinoma, NOS",TCGA-B0-5698_demographic,stage i,28438.0,8310/3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006.0,not reported,not reported,2019-07-31T23:28:08.600261-05:00,"Clear cell adenocarcinoma, NOS",TCGA-CZ-5454_demographic,stage iv,23083.0,8310/3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004.0,not reported,not reported,2019-07-31T23:10:53.767164-05:00,"Clear cell adenocarcinoma, NOS",TCGA-B0-4844_demographic,stage iv,22220.0,8310/3,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
